# Scrabu Prototype (for a List of Shipment Numbers)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

#### Configure Logger

In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')

#### Generate a list of shipment numbers

In [2]:
def generate_shipment_numbers(shipment_number=None, size=2): #340434188193324407
    logger.info("Generating shipment numbers with seed: {}".format(shipment_number))
    import numpy as np
    shipment_numbers_list = []
    for i in range(0, size):
        shipment_number_str = str(shipment_number).rjust(20, '0')
        shipment_numbers_list.append(shipment_number_str)
        shipment_number = shipment_number + 1
    logger.info("Generated {} unique shipment numbers".format(len(set(shipment_numbers_list))))
    return shipment_numbers_list

In [3]:
shipment_numbers = generate_shipment_numbers(shipment_number=340434188193323500, size=50)

05:07:48 INFO: Generating shipment numbers with seed: 340434188193323500
05:07:48 INFO: Generated 50 unique shipment numbers


#### Download the HTML content for a list of shipment numbers

In [4]:
def request(shipment_number=None, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode="):
    import requests
    import numpy as np
    logger.debug("Making HTTP request for shipment number {}".format(shipment_number))
    request_url = start_url + str(shipment_number)
    return requests.get(request_url).content

In [5]:
html_response = request(shipment_number="00340434188193323500")

#### Parsing HTML and converting it into JSON

In [6]:
def html_to_json(html):
    from lxml import etree
    import json
    logger.debug("Converting HTML to JSON")
    
    html_tree = etree.HTML(html)
    
    def clean_json(dirty_json):
        start = dirty_json.find('JSON.parse(')
        end = dirty_json.find('"),', start)
        cjson = dirty_json[start:end]
        cjson = cjson.replace('JSON.parse("', '')
        cjson = cjson.replace('\\', '')
        return cjson
    
    def find_json_element(html_tree):
        json_element = html_tree.xpath('//div')
        return str(etree.tostring(json_element[0]))
    
    dirty_json = find_json_element(html_tree)
    json_string = clean_json(dirty_json)
    return json.loads(json_string)

In [7]:
shipment_details_json = html_to_json(html_response)

#### Structure the shipment details in JSON format

In [8]:
def shipment_details(shipment_details_json, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number="00340434188193323500"):
    import datetime
    logger.debug("Preparing JSON for persistance")
    delivery_history_dict = {}
    delivery_history_dict["shipment_number"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsnummern"].get("sendungsnummer")
    delivery_history_dict["crawltime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    delivery_history_dict["url"] = start_url + str(shipment_number) 
    delivery_history_dict["events"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsverlauf"].get("events", [])
    return delivery_history_dict

In [9]:
shipment_history = shipment_details(shipment_details_json)

In [10]:
import json
def save_dictionary(shipment_history=None, filename=None):
    logger.info("Writing file {}".format(filename))
    with open(filename, 'w') as f:
        json.dump(shipment_history, f)

In [11]:
save_dictionary(shipment_history=None, filename="../data/00340434188193323500.json")

05:07:49 INFO: Writing file ../data/00340434188193323500.json


#### Main function, which goes through all shipment numbers and uses the previous methods

In [25]:
import time
def main(shipment_number=None, size=None):
    shipment_numbers = generate_shipment_numbers(shipment_number=shipment_number, size=size)
    count_saved = 0
    start_index = 0
    found_index = []
    for shipment_number in shipment_numbers:
        time.sleep(0.5)
        html_response = request(shipment_number=shipment_number)
        shipment_details_json = html_to_json(html_response)
        shipment_history = shipment_details(shipment_details_json, shipment_number=shipment_number)
        if len(shipment_history['events']) > 0:
            save_dictionary(shipment_history, filename="../data/{}.json".format(shipment_number))
            count_saved = count_saved + 1
            found_index.append(start_index)
        else:
            logger.info("No events found for shipment number {}".format(shipment_number))
        start_index = start_index + 1
        print(found_index)
        import numpy as np
        print(np.diff(found_index))
    logger.info("Saved {} files out of {}".format(count_saved, size))
    

In [29]:
main(shipment_number=340434188193323500, size=400)

05:29:48 INFO: Generating shipment numbers with seed: 340434188193323500
05:29:48 INFO: Generated 400 unique shipment numbers
05:29:49 INFO: No events found for shipment number 00340434188193323500


[]
[]


05:29:50 INFO: No events found for shipment number 00340434188193323501


[]
[]


05:29:50 INFO: No events found for shipment number 00340434188193323502


...

[9, 16, 23, 30, 47, 54, 61, 78, 85, 92, 108, 115, 122, 139, 146, 153, 160, 177, 184, 191, 207, 214, 221, 238, 245, 252, 269, 276, 283, 290, 306, 313, 320, 337, 344, 351, 368, 375, 382]
[ 7  7  7 17  7  7 17  7  7 16  7  7 17  7  7  7 17  7  7 16  7  7 17  7
  7 17  7  7  7 16  7  7 17  7  7 17  7  7]


05:34:05 INFO: No events found for shipment number 00340434188193323898


[9, 16, 23, 30, 47, 54, 61, 78, 85, 92, 108, 115, 122, 139, 146, 153, 160, 177, 184, 191, 207, 214, 221, 238, 245, 252, 269, 276, 283, 290, 306, 313, 320, 337, 344, 351, 368, 375, 382]
[ 7  7  7 17  7  7 17  7  7 16  7  7 17  7  7  7 17  7  7 16  7  7 17  7
  7 17  7  7  7 16  7  7 17  7  7 17  7  7]


05:34:06 INFO: Writing file ../data/00340434188193323899.json
05:34:06 INFO: Saved 40 files out of 400


[9, 16, 23, 30, 47, 54, 61, 78, 85, 92, 108, 115, 122, 139, 146, 153, 160, 177, 184, 191, 207, 214, 221, 238, 245, 252, 269, 276, 283, 290, 306, 313, 320, 337, 344, 351, 368, 375, 382, 399]
[ 7  7  7 17  7  7 17  7  7 16  7  7 17  7  7  7 17  7  7 16  7  7 17  7
  7 17  7  7  7 16  7  7 17  7  7 17  7  7 17]
